In [9]:
from openpyxl import load_workbook
import xlwings as xw
import os 
import pandas as pd
import numpy as np
import re
import itertools
from datetime import date,timedelta
from xlwings.constants import DeleteShiftDirection
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from dfply import *

In [10]:
# 今日日期
# Return the day of the week as an integer, where Monday is 0 and Sunday is 6. For example, date(2002, 12, 4).weekday() == 2, a Wednesday. See also isoweekday().
if date.today().weekday() == 0:
    nowDate = date.today() - timedelta(days = 3)
else:
    nowDate = date.today() - timedelta(days = 1)
todaynum = re.sub('-', '', str(nowDate))
nowYm = str(nowDate)[:7]
nowMD = nowDate.strftime("%m%d")
print(nowDate, todaynum , nowYm , nowMD)

# 本地端日報資料夾
filePath = "C:/Users/0305/Desktop/Daily PNL"
os.chdir(filePath)
if not nowYm in os.listdir(filePath):
    os.mkdir(nowYm)
filePath = filePath + '/' + nowYm
os.chdir(filePath)

if not nowMD in os.listdir(filePath):
    os.mkdir(nowMD)
filePath = filePath + '/' + nowMD
os.chdir(filePath)
#path = 'C:/Users/0305/Desktop/Daily PNL/'+nowYm+'/'+ nowMD 
#os.chdir(path)
os.listdir(filePath)

2021-01-25 20210125 2021-01 0125


['20210125_LP_swap_detail.xlsx',
 '20210125_RTvalue.xlsx',
 'AU2 90034- Rebate_20210125.xlsx',
 'AU2 90050- Rebate_20210125.xlsx',
 'AU2 90076- Rebate_20210125.xlsx',
 'AU2 90085- Rebate_20210125.xlsx',
 'AU2 90094- Rebate_20210125.xlsx',
 'B -PNL-vfx-20210125.xlsx',
 'B -PNL-vfx2-20210125.xlsx',
 'B-PNL-Mt5-20210122.xlsx',
 'B-PNL-Mt5-20210125.xlsx',
 'b_vfx2_check20210125.xlsx',
 'b_vfx_check20210125.xlsx',
 'equity_change_vfx.csv',
 'equity_change_vfx2.csv',
 'Equity_VIG_20210125.xlsx',
 'LP statement- 2021-0125.xlsx',
 'mt4_bookchange_mt4_trades.csv',
 'STP PL vfx-20210125.xlsx',
 'VANTAGE-CFD PNL-20210125.xlsx',
 'VANTAGE-stock PNL-20210125.xlsx',
 'Vanuatu_VIG PNL-20210125.xlsx',
 '~$B -PNL-vfx-20210125.xlsx']

In [155]:
# 驗證流程 
# 1. PL balance + Closed balance + Deposit != balance 誤差絕對值 > 2 
check = pd.read_excel(filePath + '/' + 'b_vfx_check' + todaynum + '.xlsx')
PL_error_value = sum(check['PL balance + Closed balance + Deposit = balance'])
p_arr = check['PL balance + Closed balance + Deposit = balance'].values
PL_error_value
#p = check$`PL balance + Closed balance + Deposit = balance`
#l = c()
type(p_arr)

numpy.ndarray

In [156]:
l = list()
while abs(PL_error_value) >2 :
    if PL_error_value <0:
        # 找min 且abs < PL_error_value
        value_index = p_arr.argsort()[len(l)+1+sum(p_arr < PL_error_value)]
        value = p_arr[value_index]
        PL_error_value = PL_error_value - value
        l.append(value_index)
    if PL_error_value >0:
        # 找MAX 且不超過值
        value_index = p_arr.argsort()[-(len(l)+1+sum(p_arr > PL_error_value))]
        value = p_arr[value_index]
        PL_error_value = PL_error_value - value
        l.append(value_index)
print(l)

[9121, 9112]


In [160]:
check['Login'][l]
error_login = check['Login'][l]
    # convert all element in list to string corresponding to mySQL statemenet 
str_l = ["'%s'" % str(elem) for elem in error_login]
idString = ", ".join(str_l)
    
    # for each db
    # for i in range(len(db)):
i = 2
mt4schema = db[i]
sql_fmt = "select * from " + mt4schema + ".mt4_daily where LOGIN in (%s);" % idString
print(sql_fmt)

select * from enfau2report.mt4_daily where LOGIN in ('690933', '691014');


In [184]:
type(df['TIME'].values[1])

numpy.datetime64

In [185]:
mt4_daily = (df >>
select(X.TIME, X.LOGIN, X.GROUP, X.BALANCE_PREV, X.PROFIT_CLOSED, X.DEPOSIT, X.BALANCE,
X.PROFIT, X.CREDIT, X.EQUITY, X.MARGIN, X.MARGIN_FREE) >>
arrange(desc(X.TIME)) >>
group_by(X.LOGIN) >>
row_slice([0,1]) )
mt4_daily.head()

,TIME,LOGIN,GROUP,BALANCE_PREV,PROFIT_CLOSED,DEPOSIT,BALANCE,PROFIT,CREDIT,EQUITY,MARGIN,MARGIN_FREE
132,2021-01-22 23:59:59,690933,M_V_90096_HKD,1304665.30,0.0,889.49,1305554.79,0.0,0.0,1305554.79,0.0,1305554.79
131,2021-01-21 23:59:59,690933,M_V_90096_HKD,1302609.78,0.0,2055.52,1304665.30,0.0,0.0,1304665.30,0.0,1304665.30
265,2021-01-22 23:59:59,691014,M_V_90096_HKD,1219476.29,0.0,831.42,1220307.71,0.0,0.0,1220307.71,0.0,1220307.71
264,2021-01-21 23:59:59,691014,M_V_90096_HKD,1217554.99,0.0,1921.30,1219476.29,0.0,0.0,1219476.29,0.0,1219476.29


In [186]:
# 資料庫設定
db_url = {
    'drivername' : 'mysql',
    'host' : "report.vi-data.net",
    'username' : "risktp",
    'password' : "reader9876",
    'port' : 3306,
    'query' : {'charset': 'BIG5'}
}


df = pd.DataFrame()
db = ['enfuk2report' , 'enfukreport' , 'enfau2report', 'enfaureport']
if len(l) > 0 :
    # connect mySQL
    engine = create_engine(URL(**db_url), encoding="BIG5")
    error_login = check['Login'][l]
    # convert all element in list to string corresponding to mySQL statemenet 
    str_l = ["'%s'" % str(elem) for elem in error_login]
    idString = ", ".join(str_l)
    
    # for each db
    for i in range(len(db)):
        mt4schema = db[i]
        sql_fmt = "select * from " + mt4schema + ".mt4_daily where LOGIN in (%s);" % idString
        sql_df = pd.read_sql_query(sql_fmt, engine)
        # pd select
        sql_df = (sql_df >>
                     select(X.TIME, X.LOGIN, X.GROUP, X.BALANCE_PREV, X.PROFIT_CLOSED, X.DEPOSIT, X.BALANCE,
                            X.PROFIT, X.CREDIT, X.EQUITY, X.MARGIN, X.MARGIN_FREE) >>
                     arrange(desc(X.TIME)) >>
                     group_by(X.LOGIN) >>
                     row_slice([0,1]) )
        df = pd.concat([df,sql_df])
print(sql_fmt)
df.tail(20)

select * from enfaureport.mt4_daily where LOGIN in ('690933', '691014');


,TIME,LOGIN,GROUP,BALANCE_PREV,PROFIT_CLOSED,DEPOSIT,BALANCE,PROFIT,CREDIT,EQUITY,MARGIN,MARGIN_FREE
132,2021-01-22 23:59:59,690933,M_V_90096_HKD,1304665.30,0.0,889.49,1305554.79,0.0,0.0,1305554.79,0.0,1305554.79
131,2021-01-21 23:59:59,690933,M_V_90096_HKD,1302609.78,0.0,2055.52,1304665.30,0.0,0.0,1304665.30,0.0,1304665.30
265,2021-01-22 23:59:59,691014,M_V_90096_HKD,1219476.29,0.0,831.42,1220307.71,0.0,0.0,1220307.71,0.0,1220307.71
264,2021-01-21 23:59:59,691014,M_V_90096_HKD,1217554.99,0.0,1921.30,1219476.29,0.0,0.0,1219476.29,0.0,1219476.29


In [12]:
# 1.修正PL balance + Closed balance + Deposit = balance
# equity change
files = [f for f in os.listdir('.') if os.path.isfile(f)]
r = re.compile("equity_change_vfx.csv")
equity_change_vfx_list = list(filter(r.match, files)) # Read Note

if len(equity_change_vfx_list)>0 :
    equity_change = pd.read_csv('equity_change_vfx.csv')
    # 更改第一天權益
    wb = xw.books('B -PNL-vfx-'+str(todaynum)+'.xlsx')
    sht = wb.sheets['equity']
    df = equity_change[equity_change['nday']==1]
    for i in range(len(df.index)):
        rownum = df.iloc[i, 1]
        value = df.iloc[i, 3:]
        # print(rownum)
        # print(type(value))
        # print(value.values)
        sht.range('D'+str(rownum)+":L"+str(rownum)).value = value.values
        sht.range('D'+str(rownum)+":L"+str(rownum)).color = (255,255,0)
    # 更改第二天權益
    sht = wb.sheets['equity_Y']
    df = equity_change[equity_change['nday']==2]
    for i in range(len(df.index)):
        rownum = df.iloc[i, 1]
        value = df.iloc[i, 3:]
        # print(rownum)
        # print(type(value))
        # print(value.values)
        sht.range('D'+str(rownum)+":L"+str(rownum)).value = value.values
        sht.range('D'+str(rownum)+":L"+str(rownum)).color = (255,255,0)
    print('PL balance + Closed balance + Deposit = balance 修正完畢')
else : 
    print('PL balance + Closed balance + Deposit = balance今日無須更動')

PL balance + Closed balance + Deposit = balance 修正完畢


In [11]:
# 檢查是否有SWAP.csv 如果有則進行swap修正
files = [f for f in os.listdir('.') if os.path.isfile(f)]
r = re.compile("vfx_swap.csv")
swap_list = list(filter(r.match, files)) # Read Note
if len(swap_list)>0:
    # 2.檢查解歸檔 修正SWAP
    swap = pd.read_csv('vfx_swap.csv')
    # 如當日有解歸檔則進行swap修正
    if swap.shape[0] > 0:
        # 開啟swap sheet 
        sht = wb.sheets['swap']
        for i in range(swap.shape[0]):
            swap.loc[i,'excel_row']
            sht.range('I'+str(swap.loc[i,'excel_row'])).value = sht.range('J'+str(swap.loc[i,'excel_row'])).value
            sht.range('I'+str(swap.loc[i,'excel_row'])).color = (255,255,0)
        print('解歸檔資料整理完畢')
        swap.head()
else: 
    print('今日無解歸檔資料')

今日無解歸檔資料


In [13]:
# 讀取RT_value表
vfxRaw = pd.read_excel(str(todaynum)+'_RTvalue.xlsx' , sheet_name= 'VFXRaw')
vfxRaw.head()

,db,Login,Group,group,CCY,Deal,Type,Profit,Comment,IO,db2,Login2,Group2,group2,Date,VIG,MNT
0,VAU1,607016,M_VIGT_USD,M,USD,30943044,balance,2.86,Transfer In from 3331,in,VAU1,3331,R_REBAT_VIG_USD,R,2021-01-25,1,1
1,VAU1,607326,M_VIG_USD,M,USD,30943531,balance,16.15,Transfer In from 3378,in,VAU1,3378,R_REBAT_VIG_USD,R,2021-01-25,1,1
2,VAU1,607326,M_VIG_USD,M,USD,30943533,balance,4.43,Transfer In from 3423,in,VAU1,3423,R_REBAT_VIG_USD,R,2021-01-25,1,1
3,VAU1,607697,M_VIG_07_USD,M,USD,30943862,balance,2500.00,Transfer In from 3479,in,VAU1,3479,R_REBAT_VIG_USD,R,2021-01-25,1,1
4,VAU1,607697,M_VIG_07_USD,M,USD,30943867,balance,400.00,Transfer In from 7631,in,VAU2,7631,R_REBAT_VIG_USD,R,2021-01-25,1,1


In [14]:
# 1.刪除GROUP含有CAN字串
# vfxRaw[vfxRaw['Group'].str.contains(r'CAN')]
# type(np.where(vfxRaw['Group'].str.contains(r'CAN'))[0])
can_rowsnumber = np.where(vfxRaw['Group'].str.contains(r'CAN'))[0]
# 2.刪除COMMENT含有/div
div_rowsnumber = np.where(vfxRaw['Comment'].str.contains(r'/div'))[0]
# 3.填補rollback group
vfxRaw['db2'].fillna(value=vfxRaw.groupby('Login')['db2'].unique(), inplace=True)
rollback = vfxRaw[vfxRaw['Comment'].str.contains(r'/Rollback')]
print(can_rowsnumber, div_rowsnumber)


[] []


In [15]:
# comment : Rollback
# vfxRaw['db2'].fillna(value=20, inplace=False)
vfxRaw[vfxRaw['Comment'].str.contains(r'/Rollback')]

,db,Login,Group,group,CCY,Deal,Type,Profit,Comment,IO,db2,Login2,Group2,group2,Date,VIG,MNT


In [16]:
# comment : PAMM 缺少group login2 
vfxRaw[(vfxRaw['Comment'].str.contains(r'PAMM')) & pd.isna(vfxRaw['Login2'])]  

,db,Login,Group,group,CCY,Deal,Type,Profit,Comment,IO,db2,Login2,Group2,group2,Date,VIG,MNT


In [18]:
# 修改RT_value表
wb = xw.books.open(str(todaynum)+'_RTvalue.xlsx')
# 使用工作表
vfx = wb.sheets['VFXRaw']

# GROUP 含有加拿大資料處理
if len(can_rowsnumber) > 0 :
    # create sheet 'CAN'
    wb.sheets.add('CAN')
    can = wb.sheets['CAN']
    # sht = wb.sheets['CAN']
    # 複製colnames至CAN SHEET
    my_values = vfx.range('A1:Q1').value 
    print(my_values)
    can.range('A1:Q1').value = my_values
    # 複製row至CAN SHEET
    for i in range(len(can_rowsnumber)):
        my_values = vfx.range('A'+str(can_rowsnumber[i]+2)+':Q'+str(can_rowsnumber[i]+2)).value 
        print(my_values)
        can.range('A'+str(i+2)+':Q'+str(i+2)).value = my_values
    # 刪除row
    for i in range(len(can_rowsnumber)):
        print(can_rowsnumber[i])
        vfx.range('A'+str(can_rowsnumber[i]+2-i)+':Q'+str(can_rowsnumber[i]+2-i)).api.Delete(DeleteShiftDirection.xlShiftUp) 

if len(div_rowsnumber) > 0:
    # 刪除row
    for i in range(len(div_rowsnumber)):
        print(div_rowsnumber[i])
        vfx.range('A'+str(div_rowsnumber[i]+2-i)+':Q'+str(div_rowsnumber[i]+2-i)).api.Delete(DeleteShiftDirection.xlShiftUp)

# Delete row 2
# sht.range('2:2').api.Delete(DeleteShiftDirection.xlShiftUp) 
wb.save()
wb.close()
# app.kill()

In [17]:
# 3. RT_value 確認完畢後複製至vfx_VIG sheet
# 修改RT_value表
wbcopy = xw.books.open(todaynum+'_RTvalue.xlsx')
wb = xw.books('B -PNL-vfx-'+str(todaynum)+'.xlsx')
# 使用工作表
vfx_Result = wbcopy.sheets['Result']
my_values = vfx_Result.range('G2:O4').options(ndim=2).value 
wb.sheets['VFX_VIG'].range('AQ10:AY12').value = my_values
print('FN Table修改完畢')
wbcopy.close()

FN Table修改完畢


In [20]:
# 4. 複製貼上Rebate與手動修正
# 讀取sales_commission & rebate 合併計算
if date.today().weekday() != 0:
    commission = pd.read_excel('VFX_sales_commission'+str(today)+'.xlsx' , sheet_name='usd')
    rebate  = pd.read_excel('VFX_rebate_report'+str(today)+'.xlsx' , sheet_name='usd')
    df = pd.concat([commission, rebate])
    df = df.groupby(['mt4Account']).agg({'inner_commission': "sum", 'outer_commission': "sum"})
    # # copy pd.df to excel
    wb = xw.books('B -PNL-vfx-'+todaynum+'.xlsx')
    sht = wb.sheets['rebate']
    rownumber_added = sht.range('A' + str(wb.sheets[0].cells.last_cell.row)).end('up').row + 1
    sht.range("A"+str(rownumber_added)).options(index=True,header=False).value=df 

In [21]:
# vfx
files = [f for f in os.listdir('.') if os.path.isfile(f)]
r = re.compile("^AU1 ")
newlist1 = list(filter(r.match, files)) # Read Note
r = re.compile("^AU2 ")
newlist2 = list(filter(r.match, files)) # Read Note
r = re.compile("^UK1 ")
newlist3 = list(filter(r.match, files)) # Read Note
r = re.compile("^UK2 ")
newlist4 = list(filter(r.match, files)) # Read Note
rebate_files = newlist1+newlist2+newlist3+newlist4
print(rebate_files)

['AU2 90034- Rebate_20210122.xlsx', 'AU2 90050- Rebate_20210122.xlsx', 'AU2 90076- Rebate_20210122.xlsx', 'AU2 90085- Rebate_20210122.xlsx', 'AU2 90094- Rebate_20210122.xlsx']


In [23]:
# 讀取需要手動更改之rebate
data = []
for i in rebate_files:
    print(i)
    login = int(i.split()[1].replace("-",""))
    wb = xw.Book(i)
    sht = wb.sheets[nowMD]
    # 刪除 rebate 含有NA的交易紀錄
    arrt = sht.range('AD:AD')[3:].value
    type(arrt)
    arrt =[x for x in arrt if x is not None]
    print('P'+str(len(arrt)+4)+':AG500')
    sht.range('P'+str(len(arrt)+4)+':AG500').clear()
    rebate_arr = [login, sht.range('M10').value, sht.range('N10').value]
    print(rebate_arr)
    data.append(rebate_arr)
    wb.close()
print(data)

AU2 90034- Rebate_20210122.xlsx
P4:AG500
[90034, 0.0, 0.0]
AU2 90050- Rebate_20210122.xlsx
P11:AG500
[90050, 0.0, 0.0]
AU2 90076- Rebate_20210122.xlsx
P4:AG500
[90076, 0.0, 0.0]
AU2 90085- Rebate_20210122.xlsx
P8:AG500
[90085, 105.60000000000001, 160.0]
AU2 90094- Rebate_20210122.xlsx
P5:AG500
[90094, 0.0, 0.0]
[[90034, 0.0, 0.0], [90050, 0.0, 0.0], [90076, 0.0, 0.0], [90085, 105.60000000000001, 160.0], [90094, 0.0, 0.0]]


In [24]:
df = pd.DataFrame(data, columns=['mt4Account', 'inner_new', 'outer_new'])
df.head()

,mt4Account,inner_new,outer_new
0,90034,0.0,0.0
1,90050,0.0,0.0
2,90076,0.0,0.0
3,90085,105.6,160.0
4,90094,0.0,0.0


In [25]:
wb = xw.books('B -PNL-vfx-'+todaynum+'.xlsx')
sht = wb.sheets['rebate']
a = sht.range('A:A').value
for i in range(len(df.index)):
    if df['mt4Account'][i] in a:
        index = a.index(df['mt4Account'][i])
        # df 取代舊值
        print(sht.range('A'+str(index+1)).value)
        sht.range('B'+str(index+1)).value = df['inner_new'][i] + df['outer_new'][i]
        # sht.range('C'+str(index+1)).value = df['outer_new'][i]
        sht.range('A'+str(index+1)+":"+'C'+str(index+1)).color = (255,255,0)
    else:
        print(str(df['mt4Account'][i]) + ' is not in list')


90034 is not in list
90050.0
90076 is not in list
90085.0
90094 is not in list


In [66]:
# app = xw.App()
# wb = app.books.open(r'\\192.168.1.20\Rmc\Data Analysis\Dropbox\Model_AUTO\VFX\B -PNL-vfx2_model.xlsx')
# Vanuatu_VIG PNL 
wb =  xw.books.open('Vanuatu_VIG PNL-'+todaynum+'.xlsx', read_only = False)
summary_vfx1_sht = wb.sheets['匯總_VFX_VIG']
summary_vfx2_sht = wb.sheets['匯總_VFX_VIG 2']
sqlcheck_vfx1 = wb.sheets['庫核對']
sqlcheck_vfx2 = wb.sheets['庫核對 2']
result = wb.sheets['result']

In [96]:
# 資料庫設定
host = "192.168.1.179" 
port = 3306
dbname = "VFX-VIG"
user = "DataTP" #
password = "Taipei_Data_123" 
engine = create_engine('mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}'.format(user=user, password=password, host=host, port=port, dbname=dbname))

In [97]:
query=f"""
SELECT 'VFX-VIG' as db,'' as login,'' as `group`, ROUND(SUM(m_pnl),10) , SUM(s_pnl),SUM(m_volume), SUM(s_volume),SUM(m_commission) ,SUM(s_commission), SUM(m_stp), SUM(s_stp),SUM(m_swap), SUM(s_swap), SUM(b_outer) + SUM(b_inner) as b_rebate, SUM(s_inner) + SUM(s_outer) as s_rebate, SUM(deposit), SUM(withdrawal) , 
date FROM `VFX-VIG`.detail 
"""+"WHERE DATE BETWEEN '"+str(nowDate)+"' AND '"+str(nowDate)+"' and db NOT REGEXP 'VAU3|VUK3' and `group` not REGEXP 'CAN' GROUP BY date;"
reb_df1 = pd.read_sql_query(query, engine)
reb_df1.head()

,db,login,group,"ROUND(SUM(m_pnl),10)",SUM(s_pnl),SUM(m_volume),SUM(s_volume),SUM(m_commission),SUM(s_commission),SUM(m_stp),SUM(s_stp),SUM(m_swap),SUM(s_swap),b_rebate,s_rebate,SUM(deposit),SUM(withdrawal),date
0,VFX-VIG,,,356985.400845,-9714.029607,2104.729803,2.591785,-27128.2919,-1582.466951,145166.2988,810.897074,-23950.893683,-637.194831,43864.31516,0.0,1.747353e+06,-208902.258492,2021-01-22


In [99]:
query=f"""
SELECT 'VFX-VIG' as db,'' as login,'' as `group`, ROUND(SUM(m_pnl),10) , SUM(s_pnl),SUM(m_volume), SUM(s_volume),SUM(m_commission) ,SUM(s_commission), SUM(m_stp), SUM(s_stp),SUM(m_swap), SUM(s_swap), SUM(b_outer) + SUM(b_inner) as b_rebate, SUM(s_inner) + SUM(s_outer) as s_rebate, SUM(deposit), SUM(withdrawal) , 
date FROM `VFX-VIG`.detail 
"""+"WHERE DATE BETWEEN '"+str(nowDate)+"' AND '"+str(nowDate)+"' and (db LIKE '%AU3' or db LIKE '%UK3') GROUP BY date;"
reb_df2 = pd.read_sql_query(query, engine)
reb_df2.head()

,db,login,group,"ROUND(SUM(m_pnl),10)",SUM(s_pnl),SUM(m_volume),SUM(s_volume),SUM(m_commission),SUM(s_commission),SUM(m_stp),SUM(s_stp),SUM(m_swap),SUM(s_swap),b_rebate,s_rebate,SUM(deposit),SUM(withdrawal),date
0,VFX-VIG,,,616868.574561,1252.780523,992.254014,1.550457,-13168.791484,-550.88821,69837.982759,166.395932,-7618.485157,-146.113383,37532.394127,0.0,505240.387363,-235029.980485,2021-01-22


In [93]:
engine.dispose()

In [83]:
# 庫核對
excel_rownumber = sqlcheck_vfx1.cells(1, "A").end('down').row + 1
sqlcheck_vfx1.range("A"+str(excel_rownumber)+":R"+str(excel_rownumber)).value = reb_df1.iloc[0].values
sqlcheck_vfx2.range("A"+str(excel_rownumber)+":R"+str(excel_rownumber)).value = reb_df2.iloc[0].values

In [75]:
sht = wb.sheets['匯總_VFX_VIG']
today_column = summary_vfx1_sht.cells(2, "F").end('right').column
today_column = today_column-1

In [76]:
chr(ord('@')+today_column)

'U'

In [32]:
# 新增一欄
sht.range(chr(ord('@')+today_column)+":"+chr(ord('@')+today_column)).api.Insert()
# 拉公式

True

In [66]:
# 拉公式
lag_col = today_column -1 
sht.range((2,lag_col),(63,lag_col)).formula_array

In [72]:
sht.range((2,today_column),(2,today_column)).value

com_error: (-2146827864, 'OLE error 0x800a01a8', None, None)

In [71]:
# Vanuatu_VIG PNL 
wbcopy = xw.books.open('VIG PNL.xlsx')
copy = wbcopy.sheets['NPL']

In [35]:
# 複製貼上
# 今日欄位
today_column = summary_vfx1_sht.cells(2, "F").end('right').column-1
sht.range((4,today_column)).value = copy.range('A1').value
sht.range((5,today_column)).value = copy.range('A2').value
sht.range((6,today_column)).value = copy.range('A3').value
sht.range((8,today_column)).value = copy.range('A4').value
sht.range((12,today_column)).value = copy.range('A5').value
sht.range((13,today_column)).value = copy.range('A6').value
sht.range((14,today_column)).value = copy.range('A7').value
sht.range((15,today_column)).value = copy.range('A8').value
sht.range((22,today_column)).value = copy.range('A9').value
sht.range((23,today_column)).value = copy.range('A10').value
sht.range((24,today_column)).value = copy.range('A11').value
sht.range((25,today_column)).value = copy.range('A12').value
sht.range((30,today_column)).value = copy.range('A13').value
sht.range((31,today_column)).value = copy.range('A14').value
sht.range((32,today_column)).value = copy.range('A15').value
sht.range((33,today_column)).value = copy.range('A16').value
sht.range((41,today_column)).value = copy.range('A17').value
sht.range((42,today_column)).value = copy.range('A18').value
sht.range((45,today_column)).value = copy.range('A19').value
sht.range((54,today_column)).value = copy.range('A20').value
sht.range((60,today_column)).value = copy.range('A21').value
sht.range((61,today_column)).value = copy.range('A22').value

In [77]:

sht = wb.sheets['匯總_VFX_VIG 2']
# 複製貼上
# 今日欄位
sht.range((4,today_column)).value = copy.range('B1').value
sht.range((5,today_column)).value = copy.range('B2').value
sht.range((6,today_column)).value = copy.range('B3').value
sht.range((8,today_column)).value = copy.range('B4').value
sht.range((12,today_column)).value = copy.range('B5').value
sht.range((13,today_column)).value = copy.range('B6').value
sht.range((14,today_column)).value = copy.range('B7').value
sht.range((15,today_column)).value = copy.range('B8').value
sht.range((22,today_column)).value = copy.range('B9').value
sht.range((23,today_column)).value = copy.range('B10').value
sht.range((24,today_column)).value = copy.range('B11').value
sht.range((25,today_column)).value = copy.range('B12').value
sht.range((30,today_column)).value = copy.range('B13').value
sht.range((31,today_column)).value = copy.range('B14').value
sht.range((32,today_column)).value = copy.range('B15').value
sht.range((33,today_column)).value = copy.range('B16').value
sht.range((41,today_column)).value = copy.range('B17').value
sht.range((42,today_column)).value = copy.range('B18').value
sht.range((45,today_column)).value = copy.range('B19').value
sht.range((54,today_column)).value = copy.range('B20').value



In [38]:
sht = wb.sheets['result']
today_column = sht.cells(3, "B").end('right').column
sht.range((3,today_column)).value

datetime.datetime(2021, 1, 22, 0, 0)

In [39]:
# result 今日欄位
sht.range((31,today_column)).value = copy.range('C1').value
sht.range((32,today_column)).value = copy.range('C2').value
sht.range((33,today_column)).value = copy.range('C3').value
sht.range((34,today_column)).value = copy.range('C4').value
sht.range((77,today_column)).value = copy.range('C5').value
sht.range((78,today_column)).value = copy.range('C6').value
sht.range((81,today_column)).value = copy.range('C7').value
sht.range((82,today_column)).value = copy.range('C8').value
sht.range((87,today_column)).value = copy.range('C9').value
sht.range((91,today_column)).value = copy.range('C10').value
sht.range((92,today_column)).value = copy.range('C11').value
sht.range((106,today_column)).value = copy.range('C12').value
sht.range((107,today_column)).value = copy.range('C13').value
sht.range((110,today_column)).value = copy.range('C14').value
sht.range((111,today_column)).value = copy.range('C15').value
sht.range((116,today_column)).value = copy.range('C16').value
sht.range((120,today_column)).value = copy.range('C17').value
sht.range((121,today_column)).value = copy.range('C18').value

In [40]:
wb.save('Vanuatu_VIG PNL-'+todaynum+'.xlsx')
wb.close()
app.quit()

NameError: name 'app' is not defined

In [212]:
# 如果有展期 計算vfx1 2 roller
# 讀取roller 
files = [f for f in os.listdir('.') if os.path.isfile(f)]
r = re.compile("roller_all")
roller = list(filter(r.match, files)) # Read Note

vfx_roller = pd.read_excel('roller_all.xlsx' , sheet_name= 'VFX')
vfx_roller.head()

type(vfxRaw['login'])
vfx_roller.dropna(subset=['login'], inplace=True)
vfx_roller

,login,group,symbol,open_time,profit,comment,Group,CCY,Profit.in.USD,Mallog,...,Unnamed: 16,M.roller.adjustment.VIG,Unnamed: 18,Result.1,value.1,Unnamed: 21,Unnamed: 22,Mal,Unnamed: 24,Thai
0,618542.0,M_VFX_PRO2_AUD,NaN,2021-01-15 00:47:28,199.51,Cash Adjustment-Rollover,M,AUD,153.680558,0.0,...,NaN,-2356.1,NaN,Mal M VIG,-446.194,NaN,NaN,888,NaN,876.0
1,621050.0,M_VIGM_P_USD,NaN,2021-01-15 00:47:28,19.40,Cash Adjustment-Rollover,M,USD,19.400000,1.0,...,NaN,NaN,NaN,VTL M VIG,0,NaN,NaN,890,NaN,882.0
2,621557.0,M_VIGM_P_USD,NaN,2021-01-15 00:47:27,19.40,Cash Adjustment-Rollover,M,USD,19.400000,1.0,...,NaN,NaN,NaN,VT M VIG,0,NaN,NaN,901,NaN,883.0
3,622767.0,M_VIG_P_AUD,NaN,2021-01-15 00:47:27,748.16,Cash Adjustment-Rollover,M,AUD,576.300166,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,903,NaN,884.0
4,624497.0,M_VIG_SGD,NaN,2021-01-15 00:47:29,77.08,Cash Adjustment-Rollover,M,SGD,57.904384,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1588,NaN,887.0


In [18]:
def stringf(x):
    return("'%s'" % str(x))

# stringf('123')

# s = ['123','445','566']
# ','.join([stringf(elem) for elem in s]) 

engine = create_engine('mysql+pymysql://{user}:{password}@{host}:{port}/allothers'.format(user=user, password=password, host=host, port=port, dbname=dbname))
idString = ','.join([stringf(elem) for elem in vfx_roller['login']]) 
sql_fmt = "SELECT db, login FROM allothers.account where login in (%s);" % idString
sql_fmt

NameError: name 'user' is not defined

In [256]:
account = pd.read_sql_query(sql_fmt, engine)
account.head()
account.drop_duplicates(subset=['db','login'],inplace = True)

vfx1 = ['VUK1','VUK2','VAU1','VAU2']
vfx2 = ['VUK3','VAU3']
vfx_roller = pd.merge(vfx_roller, account,how='left', on='login')

vfx_roller = vfx_roller.assign(
    type = lambda dataframe: vfx_roller['db'].map(lambda db: 'vfx1' if db in vfx1 else ('vfx2' if db in vfx2 else 'else')),
    VT = lambda dataframe: vfx_roller['group'].map(lambda group: bool(re.compile('VT').match(group))),
)
vfx_roller.head()

vfx_roller2 = vfx_roller[(vfx_roller.Group =='M') &( vfx_roller.VIG ==1) &( vfx_roller.type != 'else')].groupby(['VT','type']).agg({'Profit.in.USD' : sum})

vfx_roller2

KeyError: 'db'

In [254]:
vfx_roller2.iloc[0].values
vfx_roller2.iloc[1].values

array([326.31979298])

In [ ]:
vfx_CFD